# Atmospheric Model Intercomparison Project (AMIP) Validation

To investigate global stilling, we want to force the models realistic historical SST forcing in order to force the models with the observed realization. To do this we use the `amip-hist` model runs.

__Variables analyzed__
- `uas`: eastward wind component (usually 10 m) [$m \ s^{-1}$]
- `vas`: northward wind component (usually 10 m) [$m \ s^{-1}$]
- `nsws`: artificially constructed variable for net windspeed $\sqrt{uas^2 + vas^2}$

11 models are available on Andromeda (The BC Cluster) at `/data/projects/bccg/CMIP6/amip-hist/mon/uas` and `/data/projects/bccg/CMIP6/amip-hist/mon/vas` respectively. Models are at _monthly_ resolution and aggregated _yearly_ before any trend analysis is analyzed.

__Steps to connect to BC Cluster__
1. Install Remote SSH and Remote X11 extensions in VScode
2. `ssh -Y username@andromeda.bc.edu`
3. Enter password
4. `cd ~/mmfs1/data/valencig/winds-of-change`
<!-- 5. `module load python/3.9.0` $\leftarrow$ add to .tcshrc file -->

__Getting Conda up and Running__
1. `module load anaconda/2023.07-p3.11`
2. `conda init tcsh`
3. `conda create -n _envname_ python=3.11`
4. `conda activate _envname_`

This will create a conda environment in the `/mmfs1/data/_username_/.conda/envs/_envname_` directory. To automatically use this environment on login use add `conda activate _envname_` to your `.tcshrc` file.

__For a faster environment solver__
1. `conda install -n _envname_ conda-libmamba-solver`
2. `conda config --set solver libmamba`

__Export Environment__: `conda env export > environment.yml`

In [1]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
import nc_time_axis
import numpy as np
import polars as pl
import pandas as pd
import xarray as xr
import cf_xarray as cfxr
import regionmask
from glob import glob
import os
import scienceplots
plt.style.use(["science", "nature"])
%matplotlib inline

xr.set_options(keep_attrs=True)
%load_ext rich
from tqdm import tqdm

from importlib import reload

# Playing nice with CMIP6
# from xmip.preprocessing import combined_preprocessing
from xclim.ensembles import create_ensemble, ensemble_mean_std_max_min

## CMIP vs AMIP
- Using AMIP models (prescribed SSTs) to check against observational data
- Observation is only realization of the state, so use the prescribed SST to capture that single state.
- Pull in 7 amip-hist datasets, download and analyze
- Model is fundamentally flawed if AMIP doesn't capture multi-decadal trend

## Create the ensemble

In [2]:
# Get all model names
model_folders = glob('/data/projects/bccg/CMIP6/amip-hist/mon/vas/*')
model_names = [f.split('/')[-1] for f in model_folders]
model_names


[
    'IPSL-CM6A-LR',
    'IITM-ESM',
    'CanESM5',
    'CNRM-CM6-1-HR',
    'FGOALS-f3-L',
    'CAMS-CSM1-0',
    'BCC-CSM2-MR',
    'CNRM-CM6-1',
    'MIROC6',
    'CNRM-ESM2-1',
    'MRI-ESM2-0'
]

In [4]:
model = model_names[0]
# Paths for u and v wind components
# Sort to make realization indices increasing
model_uas = sorted(glob(f'/data/projects/bccg/CMIP6/amip-hist/mon/uas/{model}/*'))
model_vas = sorted(glob(f'/data/projects/bccg/CMIP6/amip-hist/mon/vas/{model}/*'))
# Get realization nunmbers
uas_realizations = [f.split('_')[-3] for f in model_uas]
vas_realizations = [f.split('_')[-3] for f in model_uas]
# Create ensembles
uas_ens = create_ensemble(model_uas, realizations=uas_realizations)
vas_ens = create_ensemble(model_vas, realizations=vas_realizations)
# Combine realizations
model_ens = xr.merge([uas_ens, vas_ens])
# Create new dimension for NSWS
model_ens['nsws'] = (model_ens.uas**2 + model_ens.vas**2)**0.5
model_ens['nsws'] = model_ens.nsws.assign_attrs(
    standard_name='near_surface_wind',
    description='Net Near-Surface Wind Speed',
    long_name='Net Near-Surface Wind Speed'
)
model_ens

<xarray.Dataset>
Dimensions:      (lat: 143, lon: 144, time: 1740, realization: 3,
                  axis_nbounds: 2)
Coordinates:
  * lat          (lat) float32 -90.0 -88.73 -87.46 -86.2 ... 87.46 88.73 90.0
  * lon          (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    height       float64 10.0
  * time         (time) datetime64[ns] 1870-01-16T12:00:00 ... 2014-12-16T12:...
  * realization  (realization) <U8 'r1i1p1f1' 'r2i1p1f1' 'r3i1p1f1'
Dimensions without coordinates: axis_nbounds
Data variables:
    time_bounds  (realization, time, axis_nbounds) float64 dask.array<chunksize=(1, 1740, 2), meta=np.ndarray>
    uas          (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1629, 143, 144), meta=np.ndarray>
    vas          (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1629, 143, 144), meta=np.ndarray>
    nsws         (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1629, 143, 144), meta=np.ndarray>
Attributes: (12/49)
    name:                  /ccc/work/cont003/gencmip6/mellull/IGCM_OUT/LMDZOR...
    Conventions:           CF-1.7 CMIP-6.2
    creation_date:         2018-07-13T14:16:44Z
    tracking_id:           hdl:21.14100/63080d2a-36b2-40a7-a8d1-e48942f4ea57
    description:           Extended AMIP run that covers 1870-2014. All natur...
    title:                 IPSL-CM6A-LR model output prepared for CMIP6 / GMM...
    ...                    ...
    parent_experiment_id:  no parent
    parent_mip_era:        no parent
    parent_source_id:      no parent
    parent_time_units:     no parent
    history:               Fri Sep 14 11:08:51 2018: ncatted -O -a parent_var...
    parent_variant_label:  no parent

## Plot zonal-mean net wind

In [ ]:
keys = ensemble_dict.keys()
n_models = len(keys)
land_region = regionmask.defined_regions.natural_earth_v5_0_0.land_110  # Land has value 0

fig = plt.figure(figsize=(14, int(n_models*2)), constrained_layout=True)
gs = fig.add_gridspec(n_models, 2, width_ratios=[5, 1])

for i, k in enumerate(tqdm(keys)):
    # Name of model
    name = k.split('.')[2]
    # map axis
    map = fig.add_subplot(gs[i, 0], projection=ccrs.Mollweide())
    # timeseries axis
    ts = fig.add_subplot(gs[i, 1])
    # Get member
    ds = ensemble_dict[k].rename({'member_id': 'realization'})  # rename to work with xclim ensembles
    ds = ds.cf.sel(T=slice('1979', None))  # 1979-2014
    # Reduce the dataset
    da = ensemble_mean_std_max_min(ds).isel(dcpp_init_year=0)
    # Extract eastward wind
    sfcWind = da['sfcWind_mean']
    # Plot map
    trend = (
        sfcWind.cf.groupby('T.year').mean()
        .polyfit('year', deg=1, skipna=True)
        .polyfit_coefficients.sel(degree=1)*10  # decadal
    )
    im = trend.plot(ax=map, vmin=-0.2, vmax=0.2, cmap='coolwarm', transform=ccrs.PlateCarree(), add_colorbar=False)
    cb = plt.colorbar(im, orientation="vertical", pad=0.15)
    cb.set_label(label='Decadal Trend [m/s]')
    # Mask data
    land_mask = land_region.mask(sfcWind.cf['X'], sfcWind.cf['Y'])
    land = sfcWind.where(land_mask == 0)
    ocean = sfcWind.where(land_mask != 0)
    # Plot time series
    land.cf.resample(T='1Y').mean().cf.mean(['time','X']).plot(y='y', label='land')
    ocean.cf.resample(T='1Y').mean().cf.mean(['time','X']).plot(y='y', label='ocean')
    # Map plot options
    map.coastlines()
    map.set_title(name)
    # Time series plot options
    ts.set_title(name)
    ts.set_ylabel('Easterly Wind [m/s]')
    ts.legend(loc='upper right')

plt.show()